In [1]:
import numpy as np
import cv2
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.optimizers import SGD, Adam
from keras.models import Sequential
from keras.layers import Convolution2D, ZeroPadding2D, MaxPooling2D
from keras.layers import normalization
import h5py
from test import *
from keras import backend as K
import time
from FaceRec.pretrained_cnn import *
import FaceRec
from FaceRec.net import *
from FaceRec.get_input import *
from keras.utils import np_utils

x,y = from_file()
x_test, y_test = test_file()

model = Sequential()
model.add(Convolution2D(64, 3, 3,input_shape=(3, 227, 227), activation='relu', name='conv1_1', border_mode='same'))
model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(64, 3, 3, activation='relu', name='conv1_2'))
model.add(MaxPooling2D((2, 2), strides=(2, 2)))

model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(128, 3, 3, activation='relu', name='conv2_1'))
model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(128, 3, 3, activation='relu', name='conv2_2'))
model.add(MaxPooling2D((2, 2), strides=(2, 2)))

model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(256, 3, 3, activation='relu', name='conv3_1'))
model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(256, 3, 3, activation='relu', name='conv3_2'))
model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(256, 3, 3, activation='relu', name='conv3_3'))
model.add(MaxPooling2D((2, 2), strides=(2, 2)))

model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(512, 3, 3, activation='relu', name='conv4_1'))
model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(512, 3, 3, activation='relu', name='conv4_2'))
model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(512, 3, 3, activation='relu', name='conv4_3'))
model.add(MaxPooling2D((2, 2), strides=(2, 2)))

model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(512, 3, 3, activation='relu', name='conv5_1'))
model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(512, 3, 3, activation='relu', name='conv5_2'))
model.add(ZeroPadding2D((1, 1)))
model.add(Convolution2D(512, 3, 3, activation='relu', name='conv5_3'))
model.add(MaxPooling2D((2, 2), strides=(2, 2)))

layer_dict = dict([(layer.name, layer) for layer in model.layers])

In [2]:
cnn = pretrained_cnn()

for k in cnn[cnn.keys()[0]]:
    for i in k:
        a = i[0][0][1][0]
        if 'conv' in a:
            weight1 = np.rollaxis(i[0][0][2][0][0],3,start=0)
            weight1 = np.rollaxis(weight1,3,start=1)
            weight2 = np.rollaxis(i[0][0][2][0][1],1,start=0)[0]
            weights = [weight1,weight2]
            layer_dict[a].set_weights(weights)
            print "Weights added to",a

In [3]:
model.add(Flatten())
model.add(Dense(output_dim=4096, activation='relu', init="uniform"))
model.add(Dense(output_dim=4096, init="uniform", activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(output_dim=7, init="uniform", activation='softmax'))

In [10]:
x,y = from_file()
x_test, y_test = test_file()

In [11]:
x = x.astype('float32')
x_test = x_test.astype('float32')
x /= 255
x_test /= 255
# x = np.subtract(x,np.average(x))
# x_test = np.subtract(x_test,np.average(x))
y = np_utils.to_categorical(y, 7)
y_test = np_utils.to_categorical(y_test, 7)

In [ ]:
sgd = SGD(lr=0.01, decay=5e-4, momentum=0.9, nesterov=True)
# adam = Adam(lr=0.1, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
model.compile(loss='categorical_crossentropy', optimizer=sgd)
print "Running training"
for i in xrange(10):
    model.fit(x, y, nb_epoch=1, batch_size=1, verbose=1, show_accuracy=True, shuffle=True)
    print model.evaluate(x_test, y_test, batch_size=1, show_accuracy=True)
    print model.predict(x_test, batch_size=1)

In [13]:
x

array([[[[ 0.04313726,  0.03529412,  0.04313726, ...,  0.09803922,
           0.10196079,  0.10196079],
         [ 0.05098039,  0.04705882,  0.04313726, ...,  0.08235294,
           0.09019608,  0.08627451],
         [ 0.05882353,  0.0627451 ,  0.04705882, ...,  0.07450981,
           0.08235294,  0.07058824],
         ..., 
         [ 0.74117649,  0.74901962,  0.69803923, ...,  0.80784315,
           0.81568629,  0.81568629],
         [ 0.72156864,  0.74901962,  0.69803923, ...,  0.81176472,
           0.81960785,  0.81960785],
         [ 0.70980394,  0.73725492,  0.69411767, ...,  0.80392158,
           0.81568629,  0.81960785]],

        [[ 0.14901961,  0.14117648,  0.1254902 , ...,  0.10196079,
           0.10588235,  0.10588235],
         [ 0.15294118,  0.14901961,  0.12941177, ...,  0.09019608,
           0.09411765,  0.10196079],
         [ 0.15294118,  0.15294118,  0.1254902 , ...,  0.09019608,
           0.09019608,  0.09019608],
         ..., 
         [ 0.74509805,  0.749019

In [14]:
x_test

array([[[[ 0.03529412,  0.04313726,  0.05098039, ...,  0.13725491,
           0.12156863,  0.09411765],
         [ 0.04705882,  0.04313726,  0.05098039, ...,  0.1254902 ,
           0.11764706,  0.09411765],
         [ 0.0627451 ,  0.05098039,  0.05882353, ...,  0.12156863,
           0.11372549,  0.09803922],
         ..., 
         [ 0.13725491,  0.16078432,  0.2       , ...,  0.17254902,
           0.17254902,  0.17254902],
         [ 0.24313726,  0.25490198,  0.26666668, ...,  0.17254902,
           0.16862746,  0.16470589],
         [ 0.23529412,  0.24705882,  0.25098041, ...,  0.16862746,
           0.17647059,  0.17647059]],

        [[ 0.03921569,  0.05098039,  0.05882353, ...,  0.16078432,
           0.15294118,  0.1254902 ],
         [ 0.04313726,  0.05098039,  0.05098039, ...,  0.14901961,
           0.14901961,  0.1254902 ],
         [ 0.05098039,  0.05098039,  0.05098039, ...,  0.14117648,
           0.14509805,  0.14117648],
         ..., 
         [ 0.12156863,  0.145098